# Chapter 24: Pipeline Orchestration

While Chapters 19-23 established individual CI practices—from triggering builds to security scanning—this chapter addresses how these components compose into sophisticated, scalable workflows. Pipeline orchestration transforms isolated scripts into enterprise-grade automation that coordinates multiple teams, handles complex dependencies, and maintains governance across hundreds of repositories.

Modern microservices architectures require pipelines that can build multiple services in dependency order, execute matrix tests across platform versions, and enforce organizational standards without duplicating configuration. Understanding orchestration patterns enables teams to scale CI from simple linear scripts to distributed systems that manage cross-repository changes and enterprise compliance requirements.

## 24.1 Sequential Pipelines

Sequential execution represents the simplest orchestration pattern, where stages execute one after another, with each stage completing before the next begins. This pattern ensures logical ordering but can create bottlenecks if stages are not carefully designed.

### Linear Stage Progression

```yaml
# GitLab CI - Sequential stages
stages:
  - validate      # Lint, format check, security scan
  - build         # Compile, package, create artifacts
  - test          # Unit, integration, e2e tests
  - security      # SAST, DAST, container scanning
  - publish       # Push to registry, create releases
  - deploy        # Deploy to environments

# Jobs within each stage execute in parallel (if no dependencies)
# But stages are strictly sequential
code_lint:
  stage: validate
  script: npm run lint

unit_tests:
  stage: test
  script: npm test
  # This waits for ALL jobs in 'build' to complete
```

**Execution Flow:**
1. All jobs in `validate` run in parallel
2. When all validate jobs succeed, `build` jobs begin
3. Pipeline fails fast—if any stage fails, subsequent stages do not execute

### Dependency-Based Sequencing

Force specific ordering within stages using `needs` (GitLab) or `dependsOn` (Azure):

```yaml
# GitLab - DAG (Directed Acyclic Graph) pipeline
build:frontend:
  stage: build
  script: npm run build:frontend

build:backend:
  stage: build
  script: npm run build:backend

test:frontend:
  stage: test
  needs: [build:frontend]  # Only waits for frontend, not backend
  script: npm run test:frontend

test:backend:
  stage: test
  needs: [build:backend]
  script: npm run test:backend

test:integration:
  stage: test
  needs: [build:frontend, build:backend]  # Waits for both
  script: npm run test:integration
```

**Visual Representation:**
```
build:frontend ────► test:frontend ───┐
                                      ▼
build:backend  ────► test:backend  ──► test:integration
```

## 24.2 Parallel Pipeline Stages

Parallel execution reduces wall-clock time by distributing independent work across multiple runners. Effective parallelization requires identifying truly independent tasks and managing resource contention.

### Horizontal Parallelization

```yaml
# GitHub Actions - Parallel jobs
jobs:
  test:
    runs-on: ubuntu-latest
    strategy:
      fail-fast: false  # Continue other jobs if one fails
      matrix:
        shard: [1, 2, 3, 4]  # 4 parallel test runners
    steps:
      - uses: actions/checkout@v4
      - run: npm ci
      - run: npm test -- --shard=${{ matrix.shard }}/4

  lint:
    runs-on: ubuntu-latest
    steps:
      - run: npm run lint

  security:
    runs-on: ubuntu-latest
    steps:
      - run: npm audit
```

**Resource Management:**
- GitHub Actions: 20 concurrent jobs per repository (free tier), scalable with paid plans
- GitLab CI: Limited by available runners
- CircleCI: Configurable parallelism per job

### Parallelization Strategies

**File-based Splitting:**
```yaml
test:
  parallel: 4
  script:
    - |
      # Get test files and split into 4 groups
      TESTS=$(find tests -name "*.test.js" | sort)
      TOTAL=$(echo "$TESTS" | wc -l)
      GROUP_SIZE=$((TOTAL / 4))
      
      # Get this job's slice
      START=$((CI_NODE_INDEX * GROUP_SIZE))
      [ $CI_NODE_INDEX -eq 4 ] && END=$TOTAL || END=$((START + GROUP_SIZE))
      
      echo "$TESTS" | sed -n "${START},${END}p" | xargs npm test
```

**Timing-Aware Splitting (CircleCI):**
```yaml
test:
  parallelism: 4
  steps:
    - run:
        command: |
          circleci tests glob "tests/**/*.test.js" | \
          circleci tests split --split-by=timings | \
          xargs npm test
```

## 24.3 Conditional Execution

Conditional execution enables pipelines to adapt to context—running different steps for pull requests versus main branch, skipping expensive tests for documentation changes, or triggering deployments only for specific tags.

### Branch and Event Conditions

```yaml
# GitLab CI - Conditional stages
deploy:production:
  stage: deploy
  script: helm upgrade myapp ./chart
  rules:
    - if: $CI_COMMIT_BRANCH == "main"
      when: manual  # Require explicit approval
    - if: $CI_COMMIT_TAG =~ /^v\d+\.\d+\.\d+/
      when: on_success  # Auto-deploy on version tags

security:scan:
  stage: security
  script: trivy image myapp:latest
  rules:
    - if: $CI_PIPELINE_SOURCE == "merge_request_event"
    - if: $CI_COMMIT_BRANCH == "main"
    - if: $CI_COMMIT_BRANCH == "develop"
```

### Path-Based Conditions

Skip unnecessary work when only specific files change:

```yaml
# GitHub Actions - Path filters
jobs:
  frontend_tests:
    if: github.event_name == 'pull_request'
    runs-on: ubuntu-latest
    steps:
      - uses: actions/checkout@v4
        with:
          fetch-depth: 0  # Full history for changed files detection
      
      - name: Check if frontend changed
        id: changed
        run: |
          if git diff --name-only HEAD^ HEAD | grep -q "^frontend/"; then
            echo "changed=true" >> $GITHUB_OUTPUT
          fi
      
      - name: Run tests
        if: steps.changed.outputs.changed == 'true'
        run: |
          cd frontend && npm test
```

### Dynamic Pipeline Generation

Generate pipeline configuration based on changed files:

```yaml
# GitLab - Dynamic child pipelines
generate-config:
  stage: generate
  script:
    - |
      # Check which services changed
      CHANGED_SERVICES=$(git diff --name-only $CI_MERGE_REQUEST_DIFF_BASE_SHA | cut -d'/' -f1 | sort -u | uniq)
      
      # Generate pipeline YAML
      echo "stages:" > generated-pipeline.yml
      echo "  - build" >> generated-pipeline.yml
      echo "" >> generated-pipeline.yml
      
      for service in $CHANGED_SERVICES; do
        if [ -d "$service" ]; then
          cat >> generated-pipeline.yml << EOF
build_$service:
  stage: build
  script:
    - cd $service && docker build .
EOF
        fi
      done
  
  artifacts:
    paths:
      - generated-pipeline.yml

trigger-child:
  stage: trigger
  trigger:
    include:
      - artifact: generated-pipeline.yml
    strategy: depend  # Wait for child pipeline
```

## 24.4 Pipeline Dependencies

Microservices and complex applications require pipelines that understand inter-service dependencies—building shared libraries before services that consume them, or coordinating cross-service integration tests.

### Inter-Service Dependencies

```yaml
# Monorepo with service dependencies
# libs/shared-utils must build before services/api and services/web

build:shared:
  stage: build
  script: 
    - cd libs/shared-utils && npm ci && npm run build
  artifacts:
    paths:
      - libs/shared-utils/dist/
    expire_in: 1 hour

build:api:
  stage: build
  needs: [build:shared]
  script:
    - cd services/api && npm ci && npm run build
  dependencies:
    - build:shared  # Download artifacts from previous job

build:web:
  stage: build
  needs: [build:shared]
  script:
    - cd services/web && npm ci && npm run build
  dependencies:
    - build:shared
```

### Cross-Repository Triggers

Trigger pipelines in other repositories when shared components change:

```yaml
# After building shared library, trigger downstream services
trigger:downstream:
  stage: trigger
  script:
    - |
      for repo in api-service web-service; do
        curl -X POST \
          -F token=$CI_JOB_TOKEN \
          -F ref=main \
          -F "variables[SHARED_LIB_VERSION]=$CI_COMMIT_SHA" \
          https://gitlab.company.com/api/v4/projects/namespace%2F$repo/trigger/pipeline
      done
  only:
    - main
    - changes:
        - libs/shared-utils/**/*
```

### Pipeline Resource Groups

Prevent concurrent execution of specific jobs to avoid resource conflicts:

```yaml
# Only one deployment to staging at a time
deploy:staging:
  stage: deploy
  resource_group: staging_environment  # Mutual exclusion
  script:
    - helm upgrade --install myapp ./chart --namespace staging
  
  # Alternative: Concurrency control in GitHub Actions
  concurrency:
    group: staging-deployment
    cancel-in-progress: false  # Queue, don't cancel
```

## 24.5 Matrix Builds

Matrix builds execute the same job with different variable combinations—testing across multiple language versions, operating systems, or dependency versions.

### Multi-Dimensional Matrices

```yaml
# GitHub Actions - Comprehensive matrix
test:
  runs-on: ${{ matrix.os }}
  strategy:
    fail-fast: false
    matrix:
      os: [ubuntu-latest, windows-latest, macos-latest]
      node-version: [18, 20, 21]
      database: [postgres, mysql, sqlite]
      exclude:
        # Exclude specific combinations
        - os: windows-latest
          database: sqlite  # SQLite not supported on Windows
      include:
        # Add specific combinations
        - os: ubuntu-latest
          node-version: 20
          database: postgres
          coverage: true  # Extra variable for this combination
  
  steps:
    - uses: actions/checkout@v4
    
    - name: Setup Node ${{ matrix.node-version }}
      uses: actions/setup-node@v4
      with:
        node-version: ${{ matrix.node-version }}
    
    - name: Setup ${{ matrix.database }}
      uses: ./.github/actions/setup-${{ matrix.database }}
    
    - name: Run tests
      run: npm test
      
    - name: Upload coverage
      if: matrix.coverage == true
      uses: codecov/codecov-action@v3
```

**Matrix Size Calculation:**
- 3 OS × 3 Node versions × 3 databases = 27 combinations
- Minus 1 exclusion = 26 parallel jobs
- Plus 1 include (already covered) = 26 total

### Platform-Specific Steps

```yaml
build:
  strategy:
    matrix:
      platform: [linux/amd64, linux/arm64, windows/amd64]
  steps:
    - name: Setup Linux
      if: startsWith(matrix.platform, 'linux')
      run: sudo apt-get install gcc-aarch64-linux-gnu
      
    - name: Setup Windows
      if: startsWith(matrix.platform, 'windows')
      run: choco install mingw
      
    - name: Build
      run: |
        docker buildx build \
          --platform ${{ matrix.platform }} \
          -t myapp:${{ matrix.platform }} \
          --push .
```

## 24.6 Pipeline Templates

Reusable templates standardize security scanning, build processes, and deployment patterns across hundreds of repositories without duplicating configuration.

### GitLab CI Templates

Include common configuration:

```yaml
# .gitlab-ci.yml in application repo
include:
  - project: 'devops/ci-templates'
    file: '/docker-build.yml'
    ref: v1.2.0  # Pin to specific version
  
  - project: 'devops/ci-templates'
    file: '/security-scan.yml'
    ref: v1.2.0

variables:
  IMAGE_NAME: myapp
  DOCKERFILE_PATH: ./Dockerfile

# Override specific job from template
build:docker:
  before_script:
    - echo "Custom step before template"
```

**Template Structure (devops/ci-templates/docker-build.yml):**
```yaml
# Template with parameters
.build:docker:
  stage: build
  image: docker:24
  services:
    - docker:24-dind
  script:
    - docker build -t $CI_REGISTRY_IMAGE/$IMAGE_NAME:$CI_COMMIT_SHA -f $DOCKERFILE_PATH .
    - docker push $CI_REGISTRY_IMAGE/$IMAGE_NAME:$CI_COMMIT_SHA

# Default implementation that repos can extend
build:docker:
  extends: .build:docker
```

### GitHub Actions Reusable Workflows

Organization-level workflow templates:

```yaml
# .github/workflows/ci.yml in application repo
name: CI

on:
  push:
    branches: [main]

jobs:
  security-scan:
    uses: company/devops-workflows/.github/workflows/security-scan.yml@v1
    with:
      image-name: myapp
      severity-threshold: high
    secrets:
      snyk-token: ${{ secrets.SNYK_TOKEN }}
  
  build-and-push:
    uses: company/devops-workflows/.github/workflows/docker-build.yml@v1
    with:
      dockerfile: ./Dockerfile
      platforms: linux/amd64,linux/arm64
    secrets:
      registry-username: ${{ secrets.REGISTRY_USER }}
      registry-password: ${{ secrets.REGISTRY_PASS }}
```

**Reusable Workflow Definition:**
```yaml
# company/devops-workflows/.github/workflows/security-scan.yml
name: Security Scan

on:
  workflow_call:
    inputs:
      image-name:
        required: true
        type: string
      severity-threshold:
        default: 'critical'
        type: string
    secrets:
      snyk-token:
        required: true

jobs:
  scan:
    runs-on: ubuntu-latest
    steps:
      - uses: actions/checkout@v4
      
      - name: Build image
        run: docker build -t ${{ inputs.image-name }} .
      
      - name: Snyk scan
        uses: snyk/actions/docker@master
        env:
          SNYK_TOKEN: ${{ secrets.snyk-token }}
        with:
          image: ${{ inputs.image-name }}
          args: --severity-threshold=${{ inputs.severity-threshold }}
```

### Template Inheritance and Overrides

```yaml
# Base template
.base:
  image: node:20-alpine
  before_script:
    - npm ci
  cache:
    key:
      files:
        - package-lock.json
    paths:
      - node_modules/

# Extend and override
test:unit:
  extends: .base
  script:
    - npm run test:unit
  
test:e2e:
  extends: .base
  image: cypress/included:latest  # Override image
  before_script:  # Override completely
    - npm ci
    - npm run build
  script:
    - npm run test:e2e
```

## 24.7 Pipeline Reusability

Beyond templates, modern CI systems provide mechanisms for sharing scripts, actions, and entire pipeline segments across projects.

### Composite Actions (GitHub)

Bundle multiple steps into a single reusable action:

```yaml
# .github/actions/setup-environment/action.yml
name: 'Setup Environment'
description: 'Setup Node, install dependencies, cache'

inputs:
  node-version:
    description: 'Node.js version'
    default: '20'
    required: false

runs:
  using: "composite"
  steps:
    - name: Setup Node
      uses: actions/setup-node@v4
      with:
        node-version: ${{ inputs.node-version }}
        cache: 'npm'
    
    - name: Install dependencies
      run: npm ci
      shell: bash
    
    - name: Verify installation
      run: npm doctor
      shell: bash
```

**Usage:**
```yaml
steps:
  - uses: actions/checkout@v4
  - uses: ./.github/actions/setup-environment
    with:
      node-version: '18'
```

### Shared Libraries (Jenkins)

Groovy libraries for common pipeline patterns:

```groovy
// vars/dockerBuild.groovy
def call(Map config) {
    pipeline {
        agent any
        
        stages {
            stage('Build') {
                steps {
                    script {
                        sh "docker build -t ${config.imageName}:${env.BUILD_NUMBER} ."
                    }
                }
            }
            
            stage('Push') {
                when {
                    branch 'main'
                }
                steps {
                    script {
                        docker.withRegistry(config.registryUrl, config.credentialsId) {
                            sh "docker push ${config.imageName}:${env.BUILD_NUMBER}"
                        }
                    }
                }
            }
        }
    }
}
```

**Usage in Jenkinsfile:**
```groovy
@Library('company-shared-library') _

dockerBuild(
    imageName: 'myapp',
    registryUrl: 'https://registry.company.com',
    credentialsId: 'registry-credentials'
)
```

## 24.8 Pipeline Governance

Governance ensures organizational compliance—security standards, naming conventions, required stages, and approval workflows—across all pipelines without manual review of every configuration file.

### Compliance as Code

Validate pipeline configuration before execution:

```yaml
# GitLab - Pipeline validation job
validate:pipeline:
  stage: .pre  # Runs before everything
  image: alpine:latest
  script:
    - apk add --no-cache yq curl
    
    # Check that security scanning is present
    - |
      if ! grep -q "security.*scan" .gitlab-ci.yml; then
        echo "ERROR: Security scanning stage missing"
        exit 1
      fi
    
    # Check that container images use specific base images only
    - |
      if grep -E "FROM (ubuntu|debian):.*" Dockerfile; then
        echo "ERROR: Use distroless or Alpine base images only"
        exit 1
      fi
    
    # Verify secrets are not hardcoded
    - |
      if grep -r "password.*=.*[^${}]" . --include="*.yml" --include="*.yaml"; then
        echo "ERROR: Potential hardcoded secret detected"
        exit 1
      fi
```

### Pipeline Policies (GitLab)

Enforce rules at the group level:

```yaml
# Group-level compliance pipeline
# Enforced on all projects in group

stages:
  - compliance
  - build

# Mandatory job in every pipeline
compliance:check:
  stage: compliance
  script:
    - echo "Validating compliance requirements..."
    - /scripts/check-approvals.sh
    - /scripts/check-security-gates.sh
  rules:
    - when: always  # Cannot be skipped

# Ensure SAST runs
include:
  - template: Security/SAST.gitlab-ci.yml
```

### Audit and Traceability

Track who triggered deployments and when:

```yaml
deploy:production:
  stage: deploy
  script:
    - |
      # Log deployment event
      curl -X POST $AUDIT_API/deployments \
        -H "Authorization: Bearer $AUDIT_TOKEN" \
        -d "{
          \"user\": \"$GITLAB_USER_LOGIN\",
          \"email\": \"$GITLAB_USER_EMAIL\",
          \"project\": \"$CI_PROJECT_NAME\",
          \"commit\": \"$CI_COMMIT_SHA\",
          \"timestamp\": \"$(date -Iseconds)\",
          \"environment\": \"production\",
          \"approval\": \"$CI_DEPLOY_FREEZE\"  # Check if during freeze period
        }"
    
    - helm upgrade --install myapp ./chart
  environment:
    name: production
    url: https://myapp.company.com
  rules:
    - if: $CI_COMMIT_BRANCH == "main"
      when: manual
      allow_failure: false
```

### Deployment Freezes

Prevent deployments during critical periods:

```yaml
deploy:production:
  stage: deploy
  rules:
    - if: $CI_DEPLOY_FREEZE == "true"
      when: never  # Block during freeze
    - if: $CI_COMMIT_BRANCH == "main"
      when: manual
  
  before_script:
    - |
      # Check if freeze is active via API
      FREEZE=$(curl -s $API/deployment-freeze/status)
      if [ "$FREEZE" == "active" ]; then
        echo "Deployment freeze is active. Cannot deploy."
        exit 1
      fi
```

---

## Chapter Summary and Preview

In this chapter, we explored pipeline orchestration patterns that scale CI from simple scripts to enterprise-grade automation systems. We examined sequential pipelines that enforce strict stage ordering, ensuring security scans complete before deployment, alongside dependency-based DAG configurations that optimize throughput by parallelizing independent work. Conditional execution patterns enable pipelines to adapt to context—skipping unnecessary tests for documentation changes, requiring manual approvals for production deployments, or triggering different workflows for hotfixes versus feature development. Matrix builds provide comprehensive coverage across multiple platform and version combinations, ensuring application compatibility without manual configuration proliferation. Pipeline templates and reusable workflows standardize organizational practices, embedding security scanning and compliance checks into every repository without duplication, while governance mechanisms enforce these standards through automated validation, mandatory stages, and audit trails that maintain traceability across the software delivery lifecycle.

**Key Takeaways:**
- Use Directed Acyclic Graph (DAG) dependencies rather than strict sequential stages to minimize pipeline duration; identify and parallelize independent jobs while maintaining necessary ordering through explicit `needs` or `dependsOn` declarations
- Implement matrix builds for testing across platform variations, but exclude invalid combinations and use `fail-fast: false` to ensure all variants are tested even if one fails, providing complete compatibility data
- Create reusable workflow templates at the organizational level to embed security scanning, compliance checks, and deployment patterns consistently across hundreds of repositories without configuration drift
- Enforce pipeline governance through compliance-as-code validation that rejects configurations lacking required security stages or using prohibited base images, preventing security debt from entering the system
- Maintain comprehensive audit trails for production deployments, capturing who triggered the deployment, what artifact was deployed, and whether it occurred during approved windows or freeze periods

**Next Chapter Preview:**
Chapter 25: Jenkins introduces the first specific CI/CD platform in detail. We will explore Jenkins architecture including master/agent topology, Pipeline DSL (both Declarative and Scripted), shared libraries for code reuse, and plugin ecosystem management. This chapter examines how to configure Jenkins for containerized builds, integrate with Kubernetes for dynamic agent provisioning, and implement security best practices including credential management and sandboxing. Understanding Jenkins provides foundational knowledge applicable to other platforms while addressing the specific configuration patterns required for self-hosted CI infrastructure that many enterprises still maintain alongside cloud-native alternatives.

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='23. code_quality_and_security_scanning.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='../5. cicd_platforms/25. jenkins.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
